In [2]:
import pandas as pd
import requests
import math
import os

In [3]:
### use this cell if you use pre computed distance matrix 
!git clone https://github.com/madalenabarcelo/project_obp_joris_3.git
%cd project_obp_joris_3

#parameters
file_path = 'distance_matrix_mini.csv'
max_loc = 5
#exact or heuristic solver, exact is slower
EXACT = False
distance_df = pd.read_csv(file_path, index_col = 0)

[WinError 2] The system cannot find the file specified: 'project_obp_joris_3'
C:\Users\daane


'git' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# # 1) Determines the problem that gets solved, basically the parameters
# file_path = 'C:/Users/daane/Downloads/Re_ project OBP/mini.csv'
# max_loc = 5
# #exact or heuristic solver, exact is slower
# EXACT = False 
# data = pd.read_csv(file_path)


# locations = []
# for i, row in data.iterrows():
#     loc = {
#         'lon': row['lon'],
#         'lat': row['lat'],
#         'name': row['name'],
#         'unique_name': f"{row['name']}_{i}"
#     }
#     locations.append(loc)

# # Add the UNIVERSAL_DEPOT location
# UNIVERSAL_DEPOT = (52.0, 5.0)
# locations.append({
#     'lon': UNIVERSAL_DEPOT[1],
#     'lat': UNIVERSAL_DEPOT[0],
#     'name': 'Universal Depot',
#     'unique_name': 'Universal_Depot'
# })

# def get_osrm_distance_submatrix(
#     src_batch, dst_batch, 
#     base_url="http://localhost:5000",
#     profile="driving"
# ):
#     """
#     Send a table request to OSRM to get the distances between
#     src_batch (sources) and dst_batch (destinations).
#     Returns a 2D list (or None on error).
#     """

#     # Combine both source and destination coordinates into one list
#     combined = src_batch + dst_batch
#     # Build the coordinate string: lon,lat;lon,lat;...
#     coords_str = ";".join(f"{loc['lon']},{loc['lat']}" for loc in combined)

#     # OSRM indices:
#     #   - sources = 0..(len(src_batch)-1)
#     #   - destinations = len(src_batch)..(len(src_batch)+len(dst_batch)-1)
#     src_indices = list(range(0, len(src_batch)))
#     dst_indices = list(range(len(src_batch), len(src_batch) + len(dst_batch)))

#     # Construct request
#     url = f"{base_url}/table/v1/{profile}/{coords_str}"
#     params = {
#         "annotations": "distance",
#         "sources": ";".join(map(str, src_indices)),
#         "destinations": ";".join(map(str, dst_indices))
#     }

#     try:
#         response = requests.get(url, params=params)
#         response.raise_for_status()
#         data = response.json()

#         # Distances is a list of lists with dimension:
#         #   len(sources) x len(destinations)
#         if "distances" in data:
#             return data["distances"]
#         else:
#             print("No distances found in OSRM response.")
#             return None

#     except requests.exceptions.RequestException as e:
#         print(f"Error with OSRM request: {e}")
#         return None


# def create_batched_distance_matrix(locations, batch_size=100):
#     """
#     Splits 'locations' into smaller sub-batches, then for each pair of
#     (source_batch, destination_batch), calls OSRM to fill in the submatrix.
#     Returns a complete distance matrix in a DataFrame with no NaN values.
#     """
#     num_locations = len(locations)
#     # All unique names in order
#     all_unique_names = [loc['unique_name'] for loc in locations]

#     # Prepare empty DataFrame
#     full_matrix = pd.DataFrame(
#         data=float("nan"),
#         index=all_unique_names,
#         columns=all_unique_names
#     )

#     # Outer loop over source batches
#     for i in range(0, num_locations, batch_size):
#         src_batch = locations[i : i + batch_size]
#         src_unique_names = [loc['unique_name'] for loc in src_batch]

#         # Inner loop over destination batches
#         for j in range(0, num_locations, batch_size):
#             dst_batch = locations[j : j + batch_size]
#             dst_unique_names = [loc['unique_name'] for loc in dst_batch]

#             # Request submatrix from OSRM
#             distances = get_osrm_distance_submatrix(src_batch, dst_batch)
#             if distances is not None:
#                 # "distances" is len(src_batch) x len(dst_batch)
#                 sub_df = pd.DataFrame(
#                     data=distances,
#                     index=src_unique_names,
#                     columns=dst_unique_names
#                 )
#                 # Update the corresponding block of the full matrix
#                 full_matrix.loc[src_unique_names, dst_unique_names] = sub_df

#     return full_matrix

# ###this is used in the file
# distance_df = create_batched_distance_matrix(locations, batch_size=100)



In [5]:
def filter_company_distances(distance_matrix, company_names, depot_name="Universal_Depot"):
    """
    Filters the distance matrix to include only the rows and columns
    for specific companies and the Universal Depot.

    Args:
        distance_matrix (pd.DataFrame): The original distance matrix as a pandas DataFrame.
        company_names (list of str): The names of the companies whose data points are to be included.
        depot_name (str): The name of the Universal Depot. Default is "Universal_Depot".

    Returns:
        pd.DataFrame: A filtered distance matrix.
    """
    # Identify the rows and columns to include (companies + depot)
    relevant_indices = [
        index for index in distance_matrix.index 
        if any(company_name in index for company_name in company_names) or depot_name in index
    ]

    # Filter the matrix
    filtered_matrix = distance_matrix.loc[relevant_indices, relevant_indices]
    
    return filtered_matrix

# Example usage (assuming your data is in a pandas DataFrame named 'distance_matrix')
filtered_matrix = filter_company_distances(distance_df, ["NextGen Technologies"])


In [6]:
from networkx import DiGraph
from vrpy import VehicleRoutingProblem
import logging

def solve_cvrp_numeric_ids(distance_matrix, depot_name="Universal_Depot"):
    """Solve a capacitated vehicle routing problem with numeric node IDs for customers.

    Args:
        distance_matrix: DataFrame representing distances between locations.
        depot_name: Name of the depot in the distance matrix.
        max_locations_per_vehicle: Maximum number of locations a vehicle can visit.

    Returns:
        A dictionary with routes and the total distance, or None if no solution is found.
    """
    logging.getLogger("vrpy").setLevel(logging.WARNING)
    G = DiGraph()
    G.add_node("Source", demand=0)
    G.add_node("Sink", demand=0)

    # Identify all matrix labels
    labels = list(distance_matrix.index)
    # Remove the depot label from "labels"
    customer_labels = [lab for lab in labels if lab != depot_name]

    # We'll map each label to an integer ID
    label_to_id = {lab: idx for idx, lab in enumerate(customer_labels)}

    # Add each customer to the graph with a demand of 1
    for lab, node_id in label_to_id.items():
        G.add_node(node_id, demand=1)

    # Edges among customers
    for i_lab, i_id in label_to_id.items():
        for j_lab, j_id in label_to_id.items():
            if i_lab != j_lab:
                cost = distance_matrix.loc[i_lab, j_lab]
                G.add_edge(i_id, j_id, cost=cost)

    # Edges from Source to each customer & customer to Sink
    for lab, node_id in label_to_id.items():
        cost_from_depot = distance_matrix.loc[depot_name, lab]
        cost_to_depot   = distance_matrix.loc[lab, depot_name]
        G.add_edge("Source", node_id, cost=cost_from_depot)
        G.add_edge(node_id, "Sink", cost=cost_to_depot)

    # Set up the VRP with capacity constraint
    vrp = VehicleRoutingProblem(G)
    vrp.source = "Source"
    vrp.sink   = "Sink"
    vrp.load_capacity = max_loc
    vrp.exact = EXACT

    try:
        vrp.solve()
    except Exception as e:
        print("Error:", e)
        return None

    return {
        "Routes": vrp.best_routes,
        "Total Distance": vrp.best_value
    }

# # Usage Example
# if __name__ == "__main__":
#     # Replace 'filtered_matrix' with your actual distance matrix DataFrame
#     result = solve_cvrp_numeric_ids(filtered_matrix, depot_name="Universal_Depot", max_locations_per_vehicle=5)
#     if result:
#         print("Routes:", result["Routes"])
#         print("Total Distance:", result["Total Distance"])
#     else:
#         print("No solution found.")


In [7]:
def unique_company_names(distance_matrix, depot_name="Universal_Depot"):
    """
    Extracts unique company names from a distance matrix, excluding the depot.

    
    Parameters:
        distance_matrix (pd.DataFrame): Distance matrix with rows and columns as node labels.
        depot_name (str): The name of the depot to exclude (default is "Universal_Depot").
    
    Returns:
        list: A sorted list of unique company names.
    """
    # Extract row/column labels
    labels = distance_matrix.index.tolist()
    
    # Extract company names by splitting at the last underscore and excluding the depot
    company_names = {label.rsplit("_", 1)[0] for label in labels if label != depot_name}
    
    # Convert to a sorted list and return
    return sorted(company_names)


In [8]:
def all_filtered_matrices():
    set_matrices = []
    list = unique_company_names(distance_df)

    for company in list:
        # Filter rows and columns for the company and the depot
        relevant_nodes = [node for node in distance_df.index if node.startswith(company)] + ['Universal_Depot']
        filtered_matrix = distance_df.loc[relevant_nodes, relevant_nodes]
        set_matrices.append(filtered_matrix)

    return set_matrices


In [9]:
def solo_routes():
    solo_results = []
    temp_matrices = all_filtered_matrices()
    for matrices in temp_matrices:
        solo_results.append(unique_company_names(matrices))
        solo_results.append(solve_cvrp_numeric_ids(matrices, depot_name="Universal_Depot"))
    return solo_results

idk = solo_routes()
idk

[['Global Group'],
 {'Routes': {1: ['Source', 4, 1, 2, 3, 0, 'Sink']},
  'Total Distance': 339942.1},
 ['NextGen Technologies'],
 {'Routes': {1: ['Source', 4, 1, 3, 0, 2, 'Sink']},
  'Total Distance': 415380.39999999997},
 ['Pioneer Networks'],
 {'Routes': {1: ['Source', 3, 2, 1, 4, 0, 'Sink']},
  'Total Distance': 567102.1},
 ['Swift Industries'],
 {'Routes': {1: ['Source', 3, 0, 2, 4, 1, 'Sink']},
  'Total Distance': 306730.2}]